<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural
## Ejercicio
*   Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
*   Sacar conclusiones de los resultados.

**IMPORTANTE:** Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

## Modelo elegido:Sistema de obtención de información con NLTK utilizando un corpus de una web


In [2]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Datos
Se consumirán los datos del artículo de wikipedia sobre el deporte "Tennis" en inglés.

In [41]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Petr%C3%B3leo')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()
article_text

'el petróleo (del griego: πετρέλαιον, lit. «aceite de roca») es una mezcla de compuestos orgánicos, principalmente hidrocarburos insolubles en agua. también es conocido como oro negro, petróleo crudo o simplemente crudo.\nse produce en el interior de la tierra, por transformación de la materia orgánica acumulada en sedimentos del pasado geológico[1]\u200b y puede acumularse en trampas geológicas naturales, que se localizan a nivel mundial[2]\u200b y de donde se extrae mediante la perforación de pozos.\nen condiciones normales de presión y temperatura es un líquido bituminoso que puede presentar gran variación en diversos parámetros como color y viscosidad (desde amarillentos y poco viscosos como la gasolina hasta líquidos negros tan viscosos que apenas fluyen), densidad (entre 0,66\xa0g/ml y 0,9785\xa0g/ml), capacidad calorífica, etc. estas variaciones se deben a la diversidad de concentraciones de los hidrocarburos que componen la mezcla, esto hace que el petróleo de cada pozo o fuent

In [40]:
# Demos un vistazo
article_text

'por mitsubishi motors | 11 mayo 2022¿estás pensando en comprar un auto nuevo? es una grandiosa idea, ya que conducir un auto puede ser un gran placer, además de las ventajas de desplazamiento. sin embargo, una parte de la propiedad de un vehículo consiste en solucionar los problemas que pueden hacerlo inseguro para ti y tus pasajeros.¿sabes cuáles son las fallas más comunes en un automóvil? en el siguiente artículo, te ayudaremos a identificar los problemas que un auto puede tener. además, te explicaremos cómo reducir estas fallas.tabla de contenidos:un problema muy común en un auto con el que te puedes encontrar son las luces traseras o delanteras rotas. una luz dañada representa una situación peligrosa para ti y para los demás conductores en la carretera, incluso puede dar lugar a una multa.en general, cuando las luces de un auto no encienden se debe a una bombilla quemada o gastada. sin embargo, el agua también puede causar daños a las luces. afortunadamente, puedes identificar est

In [42]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 31804


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [43]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres:
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!   .'

In [44]:
# Demos un vistazo
text

'el petróleo (del griego: πετρέλαιον, lit. «aceite de roca») es una mezcla de compuestos orgánicos, principalmente hidrocarburos insolubles en agua. también es conocido como oro negro, petróleo crudo o simplemente crudo. se produce en el interior de la tierra, por transformación de la materia orgánica acumulada en sedimentos del pasado geológico \u200b y puede acumularse en trampas geológicas naturales, que se localizan a nivel mundial \u200b y de donde se extrae mediante la perforación de pozos. en condiciones normales de presión y temperatura es un líquido bituminoso que puede presentar gran variación en diversos parámetros como color y viscosidad (desde amarillentos y poco viscosos como la gasolina hasta líquidos negros tan viscosos que apenas fluyen), densidad (entre 0,66 g/ml y 0,9785 g/ml), capacidad calorífica, etc. estas variaciones se deben a la diversidad de concentraciones de los hidrocarburos que componen la mezcla, esto hace que el petróleo de cada pozo o fuente sea distin

In [45]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 31660


### 3 - Dividir el texto en sentencias y en palabras

In [46]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [47]:
# Demos un vistazo
corpus[:10]

['el petróleo (del griego: πετρέλαιον, lit.',
 '«aceite de roca») es una mezcla de compuestos orgánicos, principalmente hidrocarburos insolubles en agua.',
 'también es conocido como oro negro, petróleo crudo o simplemente crudo.',
 'se produce en el interior de la tierra, por transformación de la materia orgánica acumulada en sedimentos del pasado geológico \u200b y puede acumularse en trampas geológicas naturales, que se localizan a nivel mundial \u200b y de donde se extrae mediante la perforación de pozos.',
 'en condiciones normales de presión y temperatura es un líquido bituminoso que puede presentar gran variación en diversos parámetros como color y viscosidad (desde amarillentos y poco viscosos como la gasolina hasta líquidos negros tan viscosos que apenas fluyen), densidad (entre 0,66 g/ml y 0,9785 g/ml), capacidad calorífica, etc.',
 'estas variaciones se deben a la diversidad de concentraciones de los hidrocarburos que componen la mezcla, esto hace que el petróleo de cada poz

In [48]:
# Demos un vistazo
words[:20]

['el',
 'petróleo',
 '(',
 'del',
 'griego',
 ':',
 'πετρέλαιον',
 ',',
 'lit',
 '.',
 '«',
 'aceite',
 'de',
 'roca',
 '»',
 ')',
 'es',
 'una',
 'mezcla',
 'de']

In [49]:
print("Vocabulario:", len(words))

Vocabulario: 5580


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [50]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar

    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [52]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

In [53]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

<ipython-input-53-802801f2853a>:9: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Q: Como se fracciona el petroleo
A: el petróleo natural no se usa como se extrae de la naturaleza, sino que se separa en mezclas más simples de hidrocarburos que tienen usos específicos.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


Q: Naftas livianas
A: para incrementar el octanaje de naftas y gasolinas se utilizan los siguientes métodos: es la reformación de la estructura molecular de las naftas.
Q: Kerosene para industria
A: ​ la industria petrolera clasifica el petróleo crudo según su lugar de origen (p. ej.
Q: gasoil liviano
A: relacionándolo con su gravedad api el american petroleum institute clasifica el petróleo en "liviano", "mediano", "pesado" y "extrapesado": ​ predominan los hidrocarburos saturados o parafínicos.
Q: componente de gasolinas
A: proceso para la producción de un componente de gasolinas de alto octanaje por síntesis de butilenos con isobutano.
Q: componente de gasolinas
A: proceso para la producción de un componente de gasolinas de alto octanaje por síntesis de butilenos con isobutano.
Q: componente de gasolinas
A: proceso para la producción de un componente de gasolinas de alto octanaje por síntesis de butilenos con isobutano.
Q: Desarrollo Económico OCDE
A: es importante señalar, que deri